In [6]:
from tokenizers import BertWordPieceTokenizer
import pandas as pd
import os
import string
import re

In [7]:
PATH = 'C:/Users/bill/Documents/projects/data/chatbot'

In [8]:
def cleanse(text):
    text = ''.join([ x for x in str(text).lower() if x in string.printable ])
    text = re.sub(r'\b\d+\b', '', text)
    text = text.replace('\n', ' ')
    text = re.sub(r'[#`\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;$,?\'%]', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

data = pd.read_csv(os.path.join(PATH, 'cnv_2019'), sep='\t', names=[
    'rpt_mnth', 'cnv_id', 'msg_id', 'agent', 'msg_type', 'channel', 
    'creat_ts', 'text', 'intent', 'score', 'resp_cd'
])
data = data[data['agent'].isin(['coremobile', 'ccp'])]['text'].dropna().apply(cleanse).drop_duplicates().reset_index(drop=True)
data.to_csv(os.path.join(PATH, 'tokenizer/sample'), index=False, header=False)

In [9]:
!head $PATH/tokenizer/sample

i ordered authorized users on my platinum card. want to see when they expect to receive it. if tracking is available
tracking for additional cards platinum
other card has not been mailed yet you will be notified once its mailed.
hi alan thanks for contacting us could you please elaborate your query so that i can help you with it
i see one of the card has been mailed on jan under rush delivery maximum time is working days however you will receive it soon as its under rush delivery.
no chat with a customer care professional
hi joshua i am sorry to hear about your decision to cancel the card i am here to honor your request please help me with the reason for the cancellation of the card.
cancel a card
hi shashikant thank you for messaging us. i can hep you with card cancellation. if you don t mind may i know the reason of cancellation 
im no longer going to use the card


In [10]:
tokenizer = BertWordPieceTokenizer()
tokenizer.train(files=os.path.join(PATH, 'tokenizer/sample'), vocab_size=30000, min_frequency=3)

In [11]:
tokenizer.save(os.path.join(PATH, 'tokenizer'))

['C:/Users/bill/Documents/projects/data/chatbot\\tokenizer\\vocab.txt']

In [12]:
tokenizer = BertWordPieceTokenizer(os.path.join(PATH, 'tokenizer/vocab.txt'))

In [24]:
from transformers import DistilBertConfig

config = DistilBertConfig(
    vocab_size=30000,
    max_position_embeddings=512,
    n_heads=6,
    n_layers=3
)

In [25]:
from transformers import LineByLineTextDataset, DistilBertTokenizer, DistilBertForMaskedLM

tokenizer = DistilBertTokenizer.from_pretrained(os.path.join(PATH, 'tokenizer/vocab.txt'), do_lower_case=True, max_len=512)
#model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM(config=config).from_pretrained('distilbert-base-uncased')

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=os.path.join(PATH, 'tokenizer/sample'),
    block_size=128,
)

Calling DistilBertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [26]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(PATH, 'distilbert'),
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=32,
    save_steps=10000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [28]:
trainer.train()

..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


{"learning_rate": 4.997971108586268e-05, "loss": 6.128949224472046, "step": 500}
{"learning_rate": 4.9959422171725374e-05, "loss": 5.802680884361267, "step": 1000}
{"learning_rate": 4.9939133257588053e-05, "loss": 5.359263319015503, "step": 1500}
{"learning_rate": 4.9918844343450746e-05, "loss": 4.659339546203613, "step": 2000}
{"learning_rate": 4.9898555429313425e-05, "loss": 4.230475105762482, "step": 2500}
{"learning_rate": 4.987826651517611e-05, "loss": 3.9149328083992003, "step": 3000}
{"learning_rate": 4.98579776010388e-05, "loss": 3.7832697257995607, "step": 3500}
{"learning_rate": 4.9837688686901476e-05, "loss": 3.568826804637909, "step": 4000}
{"learning_rate": 4.981739977276416e-05, "loss": 3.4469726157188414, "step": 4500}
{"learning_rate": 4.979711085862685e-05, "loss": 3.3241396498680116, "step": 5000}
{"learning_rate": 4.9776821944489534e-05, "loss": 3.2045674166679383, "step": 5500}
{"learning_rate": 4.975653303035222e-05, "loss": 3.1618262119293212, "step": 6000}
{"lear

C:\Users\bill\software\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"learning_rate": 4.957393280311638e-05, "loss": 2.7053635318279268, "step": 10500}
{"learning_rate": 4.955364388897906e-05, "loss": 2.621333956718445, "step": 11000}
{"learning_rate": 4.953335497484175e-05, "loss": 2.643410466194153, "step": 11500}
{"learning_rate": 4.951306606070443e-05, "loss": 2.6186161227226257, "step": 12000}
{"learning_rate": 4.9492777146567124e-05, "loss": 2.596000167131424, "step": 12500}
{"learning_rate": 4.94724882324298e-05, "loss": 2.5665775735378267, "step": 13000}
{"learning_rate": 4.945219931829249e-05, "loss": 2.552228819847107, "step": 13500}
{"learning_rate": 4.9431910404155175e-05, "loss": 2.5118647100925444, "step": 14000}
{"learning_rate": 4.9411621490017854e-05, "loss": 2.522433589696884, "step": 14500}
{"learning_rate": 4.939133257588054e-05, "loss": 2.5040662636756896, "step": 15000}
{"learning_rate": 4.9371043661743226e-05, "loss": 2.4685007209777834, "step": 15500}
{"learning_rate": 4.935075474760591e-05, "loss": 2.432910655260086, "step": 16

{"learning_rate": 4.7585619217659474e-05, "loss": 1.9016480882167817, "step": 59500}
{"learning_rate": 4.756533030352216e-05, "loss": 1.9052011079788207, "step": 60000}
{"learning_rate": 4.754504138938484e-05, "loss": 1.9043810179233551, "step": 60500}
{"learning_rate": 4.7524752475247525e-05, "loss": 1.9105456326007844, "step": 61000}
{"learning_rate": 4.750446356111021e-05, "loss": 1.911097437620163, "step": 61500}
{"learning_rate": 4.74841746469729e-05, "loss": 1.911690155506134, "step": 62000}
{"learning_rate": 4.746388573283558e-05, "loss": 1.880244991183281, "step": 62500}
{"learning_rate": 4.744359681869827e-05, "loss": 1.866886534333229, "step": 63000}
{"learning_rate": 4.742330790456095e-05, "loss": 1.8720847388505937, "step": 63500}
{"learning_rate": 4.7403018990423634e-05, "loss": 1.8840956897735597, "step": 64000}
{"learning_rate": 4.738273007628632e-05, "loss": 1.8646244925260544, "step": 64500}
{"learning_rate": 4.7362441162149e-05, "loss": 1.8871823093891145, "step": 650

...

{"learning_rate": 6.074500892712223e-06, "loss": 1.292912920475006, "step": 1082500}
{"learning_rate": 6.054211978574907e-06, "loss": 1.2878412790298461, "step": 1083000}
{"learning_rate": 6.033923064437591e-06, "loss": 1.2598158628344536, "step": 1083500}
{"learning_rate": 6.0136341503002765e-06, "loss": 1.263903661608696, "step": 1084000}
{"learning_rate": 5.993345236162961e-06, "loss": 1.3000698479413986, "step": 1084500}
{"learning_rate": 5.973056322025646e-06, "loss": 1.2746466190218926, "step": 1085000}
{"learning_rate": 5.95276740788833e-06, "loss": 1.2741111152768134, "step": 1085500}
{"learning_rate": 5.932478493751014e-06, "loss": 1.2962345304489136, "step": 1086000}
{"learning_rate": 5.912189579613699e-06, "loss": 1.2986977437138558, "step": 1086500}
{"learning_rate": 5.891900665476384e-06, "loss": 1.26608571267128, "step": 1087000}
{"learning_rate": 5.871611751339069e-06, "loss": 1.2718926829099655, "step": 1087500}
{"learning_rate": 5.851322837201753e-06, "loss": 1.2577878

{"learning_rate": 4.1267651355299465e-06, "loss": 1.2860627691745758, "step": 1130500}
{"learning_rate": 4.1064762213926316e-06, "loss": 1.2795404821634293, "step": 1131000}
{"learning_rate": 4.086187307255316e-06, "loss": 1.2630192734599113, "step": 1131500}
{"learning_rate": 4.065898393118001e-06, "loss": 1.2701977671980857, "step": 1132000}
{"learning_rate": 4.045609478980685e-06, "loss": 1.3126185117959976, "step": 1132500}
{"learning_rate": 4.025320564843369e-06, "loss": 1.270764562010765, "step": 1133000}
{"learning_rate": 4.0050316507060545e-06, "loss": 1.3107987071871758, "step": 1133500}
{"learning_rate": 3.984742736568739e-06, "loss": 1.249496756196022, "step": 1134000}
{"learning_rate": 3.964453822431424e-06, "loss": 1.2781783623695373, "step": 1134500}
{"learning_rate": 3.944164908294108e-06, "loss": 1.2932086094617843, "step": 1135000}
{"learning_rate": 3.923875994156793e-06, "loss": 1.2802577686309815, "step": 1135500}
{"learning_rate": 3.9035870800194774e-06, "loss": 1.2

{"learning_rate": 2.1993182924849866e-06, "loss": 1.2816464748382568, "step": 1178000}
{"learning_rate": 2.179029378347671e-06, "loss": 1.2825721714496612, "step": 1178500}
{"learning_rate": 2.1587404642103556e-06, "loss": 1.2556605181694032, "step": 1179000}
{"learning_rate": 2.1384515500730402e-06, "loss": 1.2885964673757553, "step": 1179500}
{"learning_rate": 2.118162635935725e-06, "loss": 1.249369080543518, "step": 1180000}
{"learning_rate": 2.0978737217984096e-06, "loss": 1.2701292046904564, "step": 1180500}
{"learning_rate": 2.077584807661094e-06, "loss": 1.285576274752617, "step": 1181000}
{"learning_rate": 2.0572958935237785e-06, "loss": 1.2585791773200035, "step": 1181500}
{"learning_rate": 2.037006979386463e-06, "loss": 1.2575023874044418, "step": 1182000}
{"learning_rate": 2.016718065249148e-06, "loss": 1.25357055413723, "step": 1182500}
{"learning_rate": 1.9964291511118325e-06, "loss": 1.284180515050888, "step": 1183000}
{"learning_rate": 1.976140236974517e-06, "loss": 1.25

{"learning_rate": 2.515825353027106e-07, "loss": 1.2396953089237213, "step": 1226000}
{"learning_rate": 2.3129362116539526e-07, "loss": 1.253200320005417, "step": 1226500}
{"learning_rate": 2.1100470702807986e-07, "loss": 1.2877734628915787, "step": 1227000}
{"learning_rate": 1.907157928907645e-07, "loss": 1.2784199369549751, "step": 1227500}
{"learning_rate": 1.7042687875344912e-07, "loss": 1.2779794754981995, "step": 1228000}
{"learning_rate": 1.5013796461613374e-07, "loss": 1.244050340652466, "step": 1228500}
{"learning_rate": 1.2984905047881836e-07, "loss": 1.2739269877672195, "step": 1229000}
{"learning_rate": 1.09560136341503e-07, "loss": 1.255731644630432, "step": 1229500}
{"learning_rate": 8.927122220418765e-08, "loss": 1.2779735770821572, "step": 1230000}
{"learning_rate": 6.898230806687227e-08, "loss": 1.2759363926649094, "step": 1230500}
{"learning_rate": 4.869339392955689e-08, "loss": 1.2933054921627045, "step": 1231000}
{"learning_rate": 2.8404479792241524e-08, "loss": 1.2

TrainOutput(global_step=1232200, training_loss=1.473795949356173)

In [29]:
trainer.save_model(os.path.join(PATH, 'distilbert'))